# Model Training

## i. Importing packages

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import cv2
import skimage
from skimage.transform import resize
import tensorflow as tf
from tensorflow import keras
import os

print("Import Successful")

Import Successful


In [ ]:
ii. Loading dataset